# Trajectory of Outcomes After A Distal Radius Fracture
A python version of the R analysis report.

## Environment Setup
First, install the dependencies:  
`pip install pandas numpy seaborn statsmodels plotly scikit-learn`

In [1]:
# Packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.regression.mixed_linear_model import MixedLM
import plotly.express as px
import plotly.graph_objects as go

# Set plotting style
sns.set_theme()

print("All dependencies are ready")

All dependencies are ready


# Data Import

In [2]:
# Read the CSV file with index_col=0 to use the first column as index
c1 = pd.read_csv("../data/data.csv", index_col=0)
c1 = c1.reset_index().rename(columns={'index': 'X'})  # Convert index to column named 'X'
c1.head()

,X,MRN,Age at injury,Sex,ISS,CAD,Hypertension,Osteoporosis,Diabetes,Substance abuse,...,SpecificActivities_5Y,Total_5Y,UsualActivities_5Y,Procedure,admission date,Discharge,Procedure date,Aditional procedures,Revision date,Revision procedure
0,1,15390,79,F,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ORIF,2006-08-05,2006-08-08,2006-08-07,NaN,none,none
1,2,36020,63,F,9,Present,Present,NaN,NaN,NaN,...,NaN,NaN,NaN,ORIF,2013-01-10,2013-01-10,2013-01-10,NaN,41305,Revision of IF device
2,3,45377,58,F,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ORIF,2009-11-24,2009-11-26,2009-11-25,NaN,none,none
3,4,74420,77,F,9,NaN,Present,NaN,NaN,NaN,...,0.0,5.0,10.0,ORIF,2006-04-03,2006-04-06,2006-04-05,NaN,none,none
4,5,120632,63,F,9,NaN,NaN,NaN,NaN,NaN,...,1.0,6.5,0.0,ORIF,2011-11-01,2011-11-01,2011-11-01,NaN,none,none


In [3]:
# If the column name has space, replace it with underscore.
# If the column name has special character, replace it with underscore.
c1.columns = c1.columns.str.replace(' ', '_')
c1.columns = c1.columns.str.replace('[^a-zA-Z0-9_]', '_', regex=True)
c1.head()

,X,MRN,Age_at_injury,Sex,ISS,CAD,Hypertension,Osteoporosis,Diabetes,Substance_abuse,...,SpecificActivities_5Y,Total_5Y,UsualActivities_5Y,Procedure,admission_date,Discharge,Procedure_date,Aditional_procedures,Revision_date,Revision_procedure
0,1,15390,79,F,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ORIF,2006-08-05,2006-08-08,2006-08-07,NaN,none,none
1,2,36020,63,F,9,Present,Present,NaN,NaN,NaN,...,NaN,NaN,NaN,ORIF,2013-01-10,2013-01-10,2013-01-10,NaN,41305,Revision of IF device
2,3,45377,58,F,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ORIF,2009-11-24,2009-11-26,2009-11-25,NaN,none,none
3,4,74420,77,F,9,NaN,Present,NaN,NaN,NaN,...,0.0,5.0,10.0,ORIF,2006-04-03,2006-04-06,2006-04-05,NaN,none,none
4,5,120632,63,F,9,NaN,NaN,NaN,NaN,NaN,...,1.0,6.5,0.0,ORIF,2011-11-01,2011-11-01,2011-11-01,NaN,none,none


In [4]:
# Convert binary categories to 0/1
binary_cols = ['Sex', 'CAD', 'Hypertension', 'Osteoporosis', 'Diabetes', 
               'Substance_abuse', 'Alcohol_abuse', 'Depression', 'Anxiety_disorder',
               'Psychosis', 'Malignancy', 'Stroke_TIA', 'Previous_orthopedic_trauma']

# Convert Sex F->0, M->1, and others None->0, Present->1
c1['Sex'] = (c1['Sex'] == 'M').astype(int)
for col in binary_cols[1:]:
    c1[col] = (c1[col] != 'None').astype(int)
# Special case for Revision_procedure
c1['Revision_procedure'] = (c1['Revision_procedure'] == 'Removal of  device').astype(int)

# Check the data
c1.describe()

,X,MRN,Age_at_injury,Sex,ISS,CAD,Hypertension,Osteoporosis,Diabetes,Substance_abuse,...,Pain_1Y,SpecificActivities_1Y,Total_1Y,UsualActivities_1Y,Function_5Y,Pain_5Y,SpecificActivities_5Y,Total_5Y,UsualActivities_5Y,Revision_procedure
count,447.000000,4.470000e+02,447.000000,447.000000,447.000000,447.0,447.0,447.0,447.0,447.0,...,319.000000,319.000000,319.000000,319.000000,305.000000,305.000000,305.000000,305.000000,305.000000,447.000000
mean,224.000000,2.552160e+06,51.064877,0.326622,9.138702,1.0,1.0,1.0,1.0,1.0,...,9.413793,6.896552,15.183386,4.636364,3.375410,6.311475,4.170492,9.670492,2.560656,0.085011
std,129.182042,1.062478e+06,15.957842,0.469503,1.933537,0.0,0.0,0.0,0.0,0.0,...,9.616446,10.707898,17.593061,6.912257,6.484106,8.323518,8.410742,13.788240,5.016134,0.279211
min,1.000000,1.539000e+04,16.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,112.500000,1.625768e+06,39.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,2.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,224.000000,3.024391e+06,54.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,6.000000,2.000000,9.000000,2.000000,1.000000,4.000000,1.000000,5.000000,0.000000,0.000000
75%,335.500000,3.495483e+06,63.000000,1.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,14.000000,8.000000,20.750000,6.000000,4.000000,8.000000,4.000000,12.500000,3.000000,0.000000
max,447.000000,3.991658e+06,93.000000,1.000000,27.000000,1.0,1.0,1.0,1.0,1.0,...,46.000000,58.000000,87.000000,34.000000,42.000000,41.000000,52.000000,83.000000,32.000000,1.000000


# Data Clean

In [5]:
# Remove rows where all Total scores are NA or 0
total_cols = ['Total_3M', 'Total_6M', 'Total_1Y', 'Total_5Y']
c1 = c1[~(c1[total_cols].isna().all(axis=1) | (c1[total_cols] == 0).all(axis=1))]
c1.describe()

,X,MRN,Age_at_injury,Sex,ISS,CAD,Hypertension,Osteoporosis,Diabetes,Substance_abuse,...,Pain_1Y,SpecificActivities_1Y,Total_1Y,UsualActivities_1Y,Function_5Y,Pain_5Y,SpecificActivities_5Y,Total_5Y,UsualActivities_5Y,Revision_procedure
count,423.000000,4.230000e+02,423.000000,423.000000,423.000000,423.0,423.0,423.0,423.0,423.0,...,319.000000,319.000000,319.000000,319.000000,305.000000,305.000000,305.000000,305.000000,305.000000,423.000000
mean,224.456265,2.557278e+06,51.205674,0.330969,9.122931,1.0,1.0,1.0,1.0,1.0,...,9.413793,6.896552,15.183386,4.636364,3.375410,6.311475,4.170492,9.670492,2.560656,0.087470
std,129.182766,1.056225e+06,15.882052,0.471119,1.928587,0.0,0.0,0.0,0.0,0.0,...,9.616446,10.707898,17.593061,6.912257,6.484106,8.323518,8.410742,13.788240,5.016134,0.282858
min,1.000000,1.539000e+04,17.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,112.500000,1.625768e+06,39.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,2.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,224.000000,3.024391e+06,55.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,6.000000,2.000000,9.000000,2.000000,1.000000,4.000000,1.000000,5.000000,0.000000,0.000000
75%,335.500000,3.495483e+06,63.000000,1.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,14.000000,8.000000,20.750000,6.000000,4.000000,8.000000,4.000000,12.500000,3.000000,0.000000
max,447.000000,3.991658e+06,93.000000,1.000000,27.000000,1.0,1.0,1.0,1.0,1.0,...,46.000000,58.000000,87.000000,34.000000,42.000000,41.000000,52.000000,83.000000,32.000000,1.000000


In [6]:
# Data for LME
# Filter for at least two non-NA data points
c3 = c1[c1[total_cols].isna().sum(axis=1) <= 2].copy()

# Create easyc3 with regrouped attributes
cols_to_keep = ['X', 'MRN', 'Age_at_injury', 'Sex', 'ISS', 'CAD', 'Hypertension', 
                'Osteoporosis', 'Diabetes', 'Substance_abuse', 'Alcohol_abuse',
                'Depression', 'Anxiety_disorder', 'Psychosis', 'Malignancy', 
                'Stroke_TIA', 'Previous_orthopedic_trauma', 'Revision_procedure',
                'Function_Baseline', 'Pain_Baseline', 'Total_Baseline',
                'Function_3M', 'Pain_3M', 'Total_3M', 'Function_6M', 'Pain_6M', 
                'Total_6M', 'Function_1Y', 'Pain_1Y', 'Total_1Y',
                'Function_5Y', 'Pain_5Y', 'Total_5Y']

easyc3 = c3[cols_to_keep].copy()

easyc3.describe()

,X,MRN,Age_at_injury,Sex,ISS,CAD,Hypertension,Osteoporosis,Diabetes,Substance_abuse,...,Total_3M,Function_6M,Pain_6M,Total_6M,Function_1Y,Pain_1Y,Total_1Y,Function_5Y,Pain_5Y,Total_5Y
count,377.000000,3.770000e+02,377.000000,377.000000,377.000000,377.0,377.0,377.0,377.0,377.0,...,136.000000,373.000000,373.000000,373.000000,312.000000,312.000000,312.000000,302.000000,302.000000,302.000000
mean,225.620690,2.573012e+06,51.355438,0.315650,9.196286,1.0,1.0,1.0,1.0,1.0,...,25.080882,8.088472,11.962466,20.037534,5.741987,9.298077,15.040064,3.299669,6.215232,9.498344
std,128.305225,1.046806e+06,15.855260,0.465392,1.860476,0.0,0.0,0.0,0.0,0.0,...,20.438364,9.521281,9.555648,18.358290,8.615208,9.601594,17.629288,6.365561,8.210076,13.529437
min,1.000000,1.539000e+04,17.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,114.000000,1.629372e+06,39.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,9.000000,1.000000,5.000000,6.500000,0.500000,2.000000,3.000000,0.000000,0.000000,0.000000
50%,226.000000,3.044895e+06,55.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,19.750000,4.500000,10.000000,14.500000,2.250000,6.000000,8.500000,0.750000,4.000000,5.000000
75%,333.000000,3.493755e+06,63.000000,1.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,37.125000,11.500000,17.000000,28.500000,7.500000,14.000000,20.500000,4.000000,8.000000,12.500000
max,447.000000,3.991658e+06,93.000000,1.000000,27.000000,1.0,1.0,1.0,1.0,1.0,...,88.500000,45.500000,45.000000,89.500000,42.000000,46.000000,87.000000,42.000000,41.000000,83.000000


In [7]:
# Create combined categories
easyc3['SubAbuse'] = ((easyc3['Substance_abuse'] + easyc3['Alcohol_abuse']) != 0).astype(int)
easyc3['Mental_illness'] = ((easyc3['Depression'] + easyc3['Anxiety_disorder'] + 
                            easyc3['Psychosis']) != 0).astype(int)


In [8]:
# Select final columns for easyc3
final_cols = ['X', 'MRN', 'Age_at_injury', 'Sex', 'ISS', 'CAD', 'Hypertension',
              'Osteoporosis', 'Diabetes', 'SubAbuse', 'Mental_illness',
              'Malignancy', 'Stroke_TIA', 'Previous_orthopedic_trauma', 
              'Revision_procedure', 'Function_Baseline', 'Pain_Baseline', 
              'Total_Baseline'] + [col for col in easyc3.columns 
                                  if any(x in col for x in ['_3M', '_6M', '_1Y', '_5Y'])]

easyc3 = easyc3[final_cols]

easyc3.describe()

,X,MRN,Age_at_injury,Sex,ISS,CAD,Hypertension,Osteoporosis,Diabetes,SubAbuse,...,Total_3M,Function_6M,Pain_6M,Total_6M,Function_1Y,Pain_1Y,Total_1Y,Function_5Y,Pain_5Y,Total_5Y
count,377.000000,3.770000e+02,377.000000,377.000000,377.000000,377.0,377.0,377.0,377.0,377.0,...,136.000000,373.000000,373.000000,373.000000,312.000000,312.000000,312.000000,302.000000,302.000000,302.000000
mean,225.620690,2.573012e+06,51.355438,0.315650,9.196286,1.0,1.0,1.0,1.0,1.0,...,25.080882,8.088472,11.962466,20.037534,5.741987,9.298077,15.040064,3.299669,6.215232,9.498344
std,128.305225,1.046806e+06,15.855260,0.465392,1.860476,0.0,0.0,0.0,0.0,0.0,...,20.438364,9.521281,9.555648,18.358290,8.615208,9.601594,17.629288,6.365561,8.210076,13.529437
min,1.000000,1.539000e+04,17.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,114.000000,1.629372e+06,39.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,9.000000,1.000000,5.000000,6.500000,0.500000,2.000000,3.000000,0.000000,0.000000,0.000000
50%,226.000000,3.044895e+06,55.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,19.750000,4.500000,10.000000,14.500000,2.250000,6.000000,8.500000,0.750000,4.000000,5.000000
75%,333.000000,3.493755e+06,63.000000,1.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,37.125000,11.500000,17.000000,28.500000,7.500000,14.000000,20.500000,4.000000,8.000000,12.500000
max,447.000000,3.991658e+06,93.000000,1.000000,27.000000,1.0,1.0,1.0,1.0,1.0,...,88.500000,45.500000,45.000000,89.500000,42.000000,46.000000,87.000000,42.000000,41.000000,83.000000


In [10]:
# Create long format data
id_vars = ['X', 'MRN', 'Age_at_injury', 'Sex', 'ISS', 'CAD', 'Hypertension',
           'Osteoporosis', 'Diabetes', 'SubAbuse', 'Mental_illness',
           'Malignancy', 'Stroke_TIA', 'Previous_orthopedic_trauma',
           'Revision_procedure', 'Function_Baseline', 'Pain_Baseline', 'Total_Baseline']

longc3 = pd.wide_to_long(easyc3, 
                        stubnames=['Function', 'Pain', 'Total'],
                        i=id_vars,
                        j='period',
                        suffix='_(3M|6M|1Y|5Y)',
                        sep='').reset_index()
longc3.describe()

,X,MRN,Age_at_injury,Sex,ISS,CAD,Hypertension,Osteoporosis,Diabetes,SubAbuse,...,Malignancy,Stroke_TIA,Previous_orthopedic_trauma,Revision_procedure,Function_Baseline,Pain_Baseline,Total_Baseline,Function,Pain,Total
count,1508.000000,1.508000e+03,1508.000000,1508.000000,1508.000000,1508.0,1508.0,1508.0,1508.0,1508.0,...,1508.0,1508.0,1508.0,1508.000000,1396.000000,1396.000000,1396.000000,1123.000000,1123.000000,1123.000000
mean,225.620690,2.573012e+06,51.355438,0.315650,9.196286,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.095491,0.553009,1.260745,1.813754,6.602850,9.825467,16.425646
std,128.177452,1.045763e+06,15.839471,0.464928,1.858623,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.293989,3.023098,3.793702,6.127181,9.110183,9.674829,18.003405
min,1.000000,1.539000e+04,17.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,114.000000,1.629372e+06,39.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.500000,3.000000,3.500000
50%,226.000000,3.044895e+06,55.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,2.500000,7.000000,10.500000
75%,333.000000,3.493755e+06,63.000000,1.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.000000,0.000000,0.000000,1.000000,9.000000,14.500000,23.000000
max,447.000000,3.991658e+06,93.000000,1.000000,27.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.000000,38.500000,39.000000,69.500000,45.500000,46.000000,89.500000


In [11]:
# Convert period to months
period_to_months = {'3M': 3, '6M': 6, '1Y': 12, '5Y': 60}
longc3['month'] = longc3['period'].map(period_to_months)
longc3.describe()

,X,MRN,Age_at_injury,Sex,ISS,CAD,Hypertension,Osteoporosis,Diabetes,SubAbuse,...,Stroke_TIA,Previous_orthopedic_trauma,Revision_procedure,Function_Baseline,Pain_Baseline,Total_Baseline,Function,Pain,Total,month
count,1508.000000,1.508000e+03,1508.000000,1508.000000,1508.000000,1508.0,1508.0,1508.0,1508.0,1508.0,...,1508.0,1508.0,1508.000000,1396.000000,1396.000000,1396.000000,1123.000000,1123.000000,1123.000000,0.0
mean,225.620690,2.573012e+06,51.355438,0.315650,9.196286,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.095491,0.553009,1.260745,1.813754,6.602850,9.825467,16.425646,NaN
std,128.177452,1.045763e+06,15.839471,0.464928,1.858623,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.293989,3.023098,3.793702,6.127181,9.110183,9.674829,18.003405,NaN
min,1.000000,1.539000e+04,17.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,114.000000,1.629372e+06,39.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.500000,3.000000,3.500000,NaN
50%,226.000000,3.044895e+06,55.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,0.000000,0.000000,0.000000,2.500000,7.000000,10.500000,NaN
75%,333.000000,3.493755e+06,63.000000,1.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,0.000000,0.000000,1.000000,9.000000,14.500000,23.000000,NaN
max,447.000000,3.991658e+06,93.000000,1.000000,27.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.000000,38.500000,39.000000,69.500000,45.500000,46.000000,89.500000,NaN


In [12]:
# Create lme_longc3 by dropping NA values
lme_longc3 = longc3.dropna(subset=['Total'])
lme_longc3.describe()

,X,MRN,Age_at_injury,Sex,ISS,CAD,Hypertension,Osteoporosis,Diabetes,SubAbuse,...,Stroke_TIA,Previous_orthopedic_trauma,Revision_procedure,Function_Baseline,Pain_Baseline,Total_Baseline,Function,Pain,Total,month
count,1123.000000,1.123000e+03,1123.000000,1123.000000,1123.000000,1123.0,1123.0,1123.0,1123.0,1123.0,...,1123.0,1123.0,1123.000000,1046.000000,1046.000000,1046.000000,1123.000000,1123.000000,1123.000000,0.0
mean,226.367765,2.569656e+06,52.044524,0.299199,9.180766,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.099733,0.625239,1.242830,1.868069,6.602850,9.825467,16.425646,NaN
std,130.285806,1.050690e+06,15.389876,0.458111,1.750046,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.299777,3.403572,3.819657,6.622803,9.110183,9.674829,18.003405,NaN
min,1.000000,1.539000e+04,17.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,113.000000,1.628921e+06,41.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.500000,3.000000,3.500000,NaN
50%,223.000000,3.015998e+06,55.000000,0.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,0.000000,0.000000,0.000000,2.500000,7.000000,10.500000,NaN
75%,337.000000,3.497326e+06,63.000000,1.000000,9.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,0.000000,0.000000,0.500000,9.000000,14.500000,23.000000,NaN
max,447.000000,3.991658e+06,93.000000,1.000000,27.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.000000,38.500000,39.000000,69.500000,45.500000,46.000000,89.500000,NaN


In [18]:
# count missing values
# lme_longc3.groupby('period')['Total_Baseline'].isnull().value_counts()
# count missing values
print("\nMissing values by period:")
print(lme_longc3.groupby('period')['Total_Baseline'].apply(lambda x: x.isna().sum()))



Missing values by period:
period
_1Y    19
_3M    10
_5Y    20
_6M    28
Name: Total_Baseline, dtype: int64


In [25]:
# LME Plot function

fig = plt.figure(figsize=(10, 20))

# Fit individual linear models for each subject
subjects = lme_longc3['X'].unique()
coefficients = []

for subject in subjects:
    subject_data = lme_longc3[lme_longc3['X'] == subject]
    if len(subject_data) >= 2:  # Need at least 2 points for regression
        X = subject_data['month'].values.reshape(-1, 1)
        y = subject_data['Total'].values
        try:
            slope, intercept = np.polyfit(X.ravel(), y, 1)
            coefficients.append({'subject': subject, 'slope': slope, 
                                'intercept': intercept})
        except:
            continue
    # Plot individual regression lines
coef_df = pd.DataFrame(coefficients)

plt.scatter(coef_df['intercept'], coef_df['slope'], alpha=0.5)
plt.xlabel('Intercept')
plt.ylabel('Slope')
plt.title('Individual Regression Coefficients')

plt.show() 

KeyError: 'intercept'

<Figure size 1000x2000 with 0 Axes>

In [23]:
# Create the plot
lme_plot = plot_individual_slopes()
plt.show()

KeyError: 'intercept'

<Figure size 1000x2000 with 0 Axes>